In [14]:
import pandas as pd
import numpy as np

import spacy
!python3 -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [15]:
testbcrp = pd.read_csv("/code/Final_Annotations_Pubmed.csv")
testbcrp.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
testbcrp

In [16]:
import numpy as np
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load pre-trained word2vec model (assuming it's in a .bin file)
word_vectors = KeyedVectors.load_word2vec_format('/scratch/BioWordVec_PubMed_MIMICIII_d200.vec.bin', binary=True)


In [17]:
labels = np.array(testbcrp['Annotations_final'])

In [45]:
full_extracted = pd.read_csv("Final_Scraped_Pubmed.csv")

In [5]:
full_extracted.head(10)

,Unnamed: 0,drug,context,Synonyms,PMCID,Protein,context_marked,Annotations_final
0,0,tyrosine,AlphaSpace has also guided the analysis of pro...,NaN,PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN
1,1,Chloroquine,AlphaSpace has also guided the analysis of pro...,"{'Chloroquinum', 'Aralen', 'Chloroquine', 'Chl...",PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN
2,2,Ciclosporin,AlphaSpace has also guided the analysis of pro...,"{'Ciclosporin', 'Restasis', 'Neoral', 'Sandimm...",PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN
3,3,CSCs,"Tumor Heterogeneity: Intratumor heterogeneity,...",NaN,PMC10650504,BCRP,"tumor heterogeneity: intratumor heterogeneity,...",NaN
4,7,Chloroquine,"Tumor Heterogeneity: Intratumor heterogeneity,...","{'Chloroquinum', 'Aralen', 'Chloroquine', 'Chl...",PMC10650504,BCRP,"tumor heterogeneity: intratumor heterogeneity,...",NaN
5,8,Ciclosporin,"Tumor Heterogeneity: Intratumor heterogeneity,...","{'Ciclosporin', 'Restasis', 'Neoral', 'Sandimm...",PMC10650504,BCRP,"tumor heterogeneity: intratumor heterogeneity,...",NaN
6,9,Hoechst,Efflux pumps are specialized proteins found in...,NaN,PMC10650504,BCRP,efflux pumps are specialized proteins found in...,NaN
7,11,ATP,Efflux pumps are specialized proteins found in...,NaN,PMC10650504,BCRP,efflux pumps are specialized proteins found in...,NaN
8,12,cisplatin,Efflux pumps are specialized proteins found in...,NaN,PMC10650504,BCRP,efflux pumps are specialized proteins found in...,0.0
9,13,DOX,Efflux pumps are specialized proteins found in...,NaN,PMC10650504,BCRP,efflux pumps are specialized proteins found in...,NaN


In [40]:
import numpy as np
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Flatten, Activation, RepeatVector, Permute, Multiply, Lambda, Bidirectional, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Dropout
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention

# Prepare the text data
texts = testbcrp['context_marked'].tolist()
# labels = ra['Relationship_Tag'].values

# Prepare tokenizer based on your texts
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)

print("="*20 + "1"*40)
# Manually add the special tokens to the tokenizer's word index
special_tokens = ['<e1>', '</e1>', '<e2>', '</e2>']
for token in special_tokens:
    tokenizer.word_index[token] = len(tokenizer.word_index) + 1

# Prepare the embedding matrix
embedding_dim = word_vectors.vector_size
vocab_size = len(tokenizer.word_index) + 1  # Update vocab_size after adding special tokens
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Populate the embedding matrix with word vectors, special tokens are handled separately
for word, i in tokenizer.word_index.items():
    if word in special_tokens:
        # Initialize special tokens with zero or random vectors
        embedding_matrix[i] = np.zeros(embedding_dim)  # or np.random.rand(embedding_dim)
    elif word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

# Create the embedding layer
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

# Define the attention mechanism : Simple
def attention_layer(inputs, lstm_units):

    attention = Dense(1, activation='tanh')(inputs)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units * 2)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([inputs, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(lstm_units * 2,))(sent_representation)
    return sent_representation
# Define the attention mechanism : Advanced
# def attention_layer(inputs, lstm_units, num_heads):
#     # `inputs` should have shape (batch_size, time_steps, lstm_units*2)
#     # `lstm_units` should be the number of hidden units in the LSTM layer
#     # `num_heads` is the number of attention heads

#     # Multi-head attention layer
#     attention_output, attention_weights = MultiHeadAttention(
#         num_heads=num_heads,  # Number of attention heads
#         key_dim=lstm_units * 2,  # Size of each attention head for query and key
#         value_dim=lstm_units * 2  # Size of each attention head for value
#     )(query=inputs, value=inputs, key=inputs, return_attention_scores=True)

#     # Flatten the attention output to be able to connect to Dense layers later
#     sent_representation = Flatten()(attention_output)

#     return sent_representation

# Model building
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
conv_layer = Conv1D(filters=128, kernel_size=16, activation='relu')(embedded_sequences)
max_pooling = MaxPooling1D(pool_size=4)(conv_layer)
bilstm_1 = Bidirectional(LSTM(512, return_sequences=True))(max_pooling)
dropout_1 = Dropout(0.5)(bilstm_1)  # Apply dropout to the output of the first BiLSTM layer
# Second BiLSTM layer
bilstm_2 = Bidirectional(LSTM(1024, return_sequences=True))(dropout_1)
dropout_2 = Dropout(0.5)(bilstm_2)  # Apply dropout to the output of the second BiLSTM layer
# bilstm_output = Bidirectional(LSTM(1024, return_sequences=True))(max_pooling)
attention_output = attention_layer(dropout_2, 1024)
output = Dense(1, activation='sigmoid')(attention_output)
model = Model(sequence_input, output)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

if not np.issubdtype(y_train.dtype, np.number):
    y_train = y_train.astype(np.float32)
    y_test = y_test.astype(np.float32)
    
# Convert labels to categorical (if they're not already in binary form)
# y_train = to_categorical(y_train, num_classes=3)
# y_test = to_categorical(y_test, num_classes=3)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=10
          , batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {accuracy:.4f}')

# Predict on new data
# predictions = model.predict(X_new)


====================1111111111111111111111111111111111111111
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1476)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 1476, 200)    20937000    ['input_3[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 1461, 128)    409728      ['embedding_2[0][0]']            
                                                                                                  
 max_pooling1d_2 (MaxPooling1D)  (None, 365, 128)    0           ['conv1d_2[0][0]']               
                               

In [41]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Assuming you have a trained model and a validation set X_val, y_val

# Get the predicted probabilities for the positive class
y_scores = model.predict(X_test)

# Calculate ROC curve
fpr, tpr, thresholds_roc = roc_curve(y_test, y_scores)

# Calculate precision-recall curve
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_scores)

# Find the optimal threshold
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold_roc = thresholds_roc[optimal_idx]

# Alternatively, if you care about precision-recall balance
optimal_idx_pr = np.argmax(precision + recall)
optimal_threshold_pr = thresholds_pr[optimal_idx_pr]

print(f'Optimal threshold according to ROC: {optimal_threshold_roc}')
print(f'Optimal threshold according to Precision-Recall: {optimal_threshold_pr}')


124/124 [==============================] - 23s 180ms/step
Optimal threshold according to ROC: 0.13570135831832886
Optimal threshold according to Precision-Recall: 0.004818522837013006


In [46]:
# Assuming 'new_texts' is a list of new text data you want to predict on
new_texts = full_extracted['context_marked'].to_list()  # replace this with your actual new texts
new_sequences = tokenizer.texts_to_sequences(new_texts)
max_length = max(len(seq) for seq in sequences)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length)

# Now make predictions using the model
predictions = model.predict(new_padded_sequences)

# If you need binary predictions, decide on a threshold and apply it
threshold = 0.135 # This is the most common threshold for binary classification
binary_predictions = (predictions > threshold).astype(int)

# Now 'binary_predictions' contains the final binary predictions for your new data


6622/6622 [==============================] - 1186s 179ms/step


In [91]:
threshold = 0.26 # This is the most common threshold for binary classification
# binary_predictions = (predictions > threshold).astype(int)

In [92]:
pd.Series(binary_predictions.reshape(211889,)).value_counts()

0    167218
1     44671
Name: count, dtype: int64

In [96]:
full_extracted['Predictions'] = predictions

In [118]:
Predictions = full_extracted.groupby(['drug'])['Predictions'].mean()

In [123]:
full_extracted

,Unnamed: 0,drug,context,Synonyms,PMCID,Protein,context_marked,Annotations_final,Predictions
0,0,tyrosine,AlphaSpace has also guided the analysis of pro...,NaN,PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN,0.116354
1,1,Chloroquine,AlphaSpace has also guided the analysis of pro...,"{'Chloroquinum', 'Aralen', 'Chloroquine', 'Chl...",PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN,0.115066
2,2,Ciclosporin,AlphaSpace has also guided the analysis of pro...,"{'Ciclosporin', 'Restasis', 'Neoral', 'Sandimm...",PMC10653122,ABCG2,alphaspace has also guided the analysis of pro...,NaN,0.115066
3,3,CSCs,"Tumor Heterogeneity: Intratumor heterogeneity,...",NaN,PMC10650504,BCRP,"tumor heterogeneity: intratumor heterogeneity,...",NaN,0.220331
4,7,Chloroquine,"Tumor Heterogeneity: Intratumor heterogeneity,...","{'Chloroquinum', 'Aralen', 'Chloroquine', 'Chl...",PMC10650504,BCRP,"tumor heterogeneity: intratumor heterogeneity,...",NaN,0.207196
...,...,...,...,...,...,...,...,...,...
211884,353082,Cisplatin,This is the first report to show that a copper...,"{'cisplatino', 'Cisplatin', 'Platinol'}",PMC5926870,BCRP,this is the first report to show that a copper...,NaN,0.046435
211885,353084,progesterone,The expression levels of mRNA for multidrug re...,NaN,PMC5926724,BCRP,the expression levels of mrna for multidrug re...,NaN,0.040444
211886,353085,estrogen,The expression levels of mRNA for multidrug re...,NaN,PMC5926724,BCRP,the expression levels of mrna for multidrug re...,NaN,0.029228
211887,353086,Chloroquine,The expression levels of mRNA for multidrug re...,"{'Chloroquinum', 'Aralen', 'Chloroquine', 'Chl...",PMC5926724,BCRP,the expression levels of mrna for multidrug re...,NaN,0.041686


In [124]:
result_df = full_extracted.groupby('drug').agg({
    'Predictions': 'mean',
    'PMCID': lambda x: list(x.unique())
}).reset_index()


In [126]:
result_df.to_csv("Final_Predictions.csv")